# 🧩 Notebook 03: Aggregation, Grouping & Reshaping in pandas

In [1]:
# Notebook import setup
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from scripts import utils_io, agg_utils

from scripts.agg_utils import (
    melt_summary,
    stacked_groupby_unstack,
    resample_monthly,
    groupby_summary,
    compute_approval_rate,
    pivot_table_summary
)

## Setup & Load All Cleaned Datasets

In [2]:
# Setup & Load All Cleaned Datasets
import pandas as pd
import numpy as np
from pathlib import Path

ASSETS_DIR = Path("../assets")

In [3]:
# Load cleaned datasets
superstore_df = utils_io.load_csv(ASSETS_DIR / "superstore_final.csv")
weather_df = utils_io.load_csv(ASSETS_DIR / "weather_final.csv")
loan_df = utils_io.load_csv(ASSETS_DIR / "loan_final.csv")
covid_df = utils_io.load_csv(ASSETS_DIR / "covid_final.csv")

## Superstore Aggregation

In [4]:
# Aggregating sales for regional segment
region_segment_sales = (
    superstore_df
    .pipe(
        agg_utils.groupby_summary,
        group_col=["region", "segment"],
        agg_dict={
            "sales": "sum",
            "profit": "mean"
        }
    )
    .sort_values("sales", ascending=False)
)
region_segment_sales.sort_values("sales", ascending=False)

,region,segment,sales,profit
11,west,home office,886198.06,48.843888
10,west,corporate,873557.42,51.877333
4,east,corporate,863581.90,49.331218
0,central,consumer,863023.85,49.925952
5,east,home office,862121.02,51.034858
2,central,home office,857793.13,52.672588
8,south,home office,832299.76,52.308431
3,east,consumer,829623.71,49.926880
1,central,corporate,806632.08,44.801336
6,south,consumer,799706.39,46.789138


In [5]:
# Perform grouped aggregation using .pipe()
category_stats_unflattened = (
    superstore_df
    .pipe(
        agg_utils.groupby_summary,
        group_col="category",
        agg_dict={
            "sales": ["count", "sum"],
            "discount": "mean"
        }
    )
    .sort_values(("sales", "sum"), ascending=False)
)

In [6]:
# Grouped aggregation with multiple functions and flattened output
category_stats_flattened = (
    superstore_df
    .pipe(
        agg_utils.groupby_summary,
        group_col="category",
        agg_dict={
            "sales": ["count", "sum"],
            "discount": "mean"
        }
    )
    .sort_values(("sales", "sum"), ascending=False)
)

# Flatten multi-level columns
category_stats_flattened.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in category_stats_flattened.columns.values
]

In [7]:
# Sales by Category
sales_by_category = (
    superstore_df
    .pipe(agg_utils.groupby_summary,
          group_col="category",
          agg_dict={
              "sales": "sum",
              "profit": "sum",
              "quantity": "sum"
          })
)

In [8]:
# Monthly Sales Trends
superstore_monthly = (
    superstore_df
    .pipe(agg_utils.resample_monthly, date_col="order_date", metrics_dict={
        "sales": "sum",
        "profit": "sum"
    })
)

/Users/satvikpraveen/Desktop/Libraries/PandasPlayground/scripts/agg_utils.py:130: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample("M").agg(metrics_dict).reset_index()


### Pivoting and Melting

In [9]:
# Pivot Sales by Region and Category
sales_pivot = agg_utils.pivot_table_summary(
    df=superstore_df,
    index="region",
    columns="category",
    values="sales",
    aggfunc="sum"
).fillna(0)

# Display
sales_pivot

category,furniture,office supplies,technology
region,,,
central,687722.13,933173.95,906552.98
east,692265.37,919657.19,943404.07
south,658185.83,879804.95,879419.63
west,697185.44,966636.72,893666.45


In [10]:
# Melt pivoted table back into long format
sales_melt = melt_summary(
    df=sales_pivot,
    id_vars="region",
    var_name="category",
    value_name="sales"
)

# Display
sales_melt.head()

,region,category,sales
0,central,furniture,687722.13
1,east,furniture,692265.37
2,south,furniture,658185.83
3,west,furniture,697185.44
4,central,office supplies,933173.95


In [11]:
# Segment vs. Sub-Category Pivot (Total Sales)

segment_subcat_sales = agg_utils.pivot_table_summary(
    df=superstore_df,
    index="segment",
    columns="sub_category",
    values="sales",
    aggfunc="sum"
)

# Display
segment_subcat_sales.head()


sub_category,accessories,binders,bookcases,chairs,copiers,labels,machines,paper,pens,phones,tables
segment,,,,,,,,,,,
consumer,320933.21,287924.74,308257.76,330217.86,274817.26,291569.52,280490.10,319812.60,289097.77,290978.62,295987.64
corporate,295639.87,320262.62,297449.30,285728.05,301284.87,308506.68,305633.35,275687.40,330747.27,310094.11,298142.14
home office,302439.45,334352.72,315803.17,280896.12,317103.06,319985.13,322414.18,319069.63,302256.73,301215.05,322876.73


### Stack/Unstack (MultiIndex Example)

In [12]:
# Multi-index groupby with unstacked category-level
stacked_sales = stacked_groupby_unstack(
    df=superstore_df,
    group_cols=["region", "segment", "category"],
    value_col="sales",
    unstack_col="category",
    fill_value=0
)

# Display
stacked_sales.head()


category             furniture  office supplies  technology
region  segment                                            
central consumer     239674.31        311743.17   311606.37
        corporate    199704.27        303157.98   303769.83
        home office  248343.55        318272.80   291176.78
east    consumer     247109.45        281449.78   301064.48
        corporate    224271.92        324336.77   314973.21

### Time Series Aggregation (e.g., Monthly Sales)

In [13]:
# Ensure dates are datetime
superstore_df["order_date"] = pd.to_datetime(superstore_df["order_date"])

# Monthly Aggregation of Sales
monthly_sales = resample_monthly(
    df=superstore_df,
    date_col="order_date",
    metrics_dict={"sales": "sum"}
)

# Display
monthly_sales.head()


/Users/satvikpraveen/Desktop/Libraries/PandasPlayground/scripts/agg_utils.py:130: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample("M").agg(metrics_dict).reset_index()


,order_date,sales
0,2020-01-31,36627.07
1,2020-02-29,26120.18
2,2020-03-31,33591.53
3,2020-04-30,24267.89
4,2020-05-31,30944.10


### Save Aggregated Output

In [14]:
# Save aggregated output
utils_io.save_csv(region_segment_sales, "../assets/superstore_region_segment_sales.csv")
utils_io.save_csv(monthly_sales, ASSETS_DIR / "superstore_monthly_sales.csv")
utils_io.save_csv(sales_by_category, "../assets/superstore_agg_category.csv")
utils_io.save_csv(superstore_monthly, "../assets/superstore_monthly_sales.csv")
utils_io.save_csv(category_stats_unflattened, "../assets/superstore_category_stats_unflattened.csv")
utils_io.save_csv(category_stats_flattened, "../assets/superstore_category_stats_flattened.csv")
utils_io.save_csv(sales_pivot.reset_index(), "../assets/superstore_sales_pivot_region_category.csv")
utils_io.save_csv(sales_melt, "../assets/superstore_sales_pivot_melted.csv")
utils_io.save_csv(stacked_sales, "../assets/superstore_sales_stacked_by_category.csv")
utils_io.save_csv(monthly_sales, "../assets/superstore_monthly_sales.csv")
utils_io.save_csv(segment_subcat_sales.reset_index(), "../assets/superstore_sales_pivot_segment_subcategory.csv")

## Weather Data (weather_final.csv)

In [15]:
# Average Temperature and Humidity by Weather Condition
condition_stats = (
    weather_df
    .pipe(
        agg_utils.groupby_summary,
        group_col="condition",
        agg_dict={
            "temperature_c": "mean",
            "humidity": "mean",
            "date": "count"
        }
    )
    .sort_values("temperature_c", ascending=False)
)

# Rename the 'date_count' column for clarity
condition_stats.rename(columns={"date": "count_days"}, inplace=True)

# Display
condition_stats.head()


,condition,temperature_c,humidity,count_days
3,storm,15.087685,65.235961,2030
4,sunny,14.810152,64.503553,1970
2,snow,14.737024,63.581809,2023
1,rain,14.188318,64.093656,1986
0,cloudy,13.833752,64.275741,1991


In [16]:
# Monthly Weather Trends
weather_df["date"] = pd.to_datetime(weather_df["date"])

# Monthly Weather Trends
weather_monthly = agg_utils.resample_monthly(
    df=weather_df,
    date_col="date",
    metrics_dict={
        "temperature_c": "mean",
        "humidity": "mean"
    }
)

# Display
weather_monthly.head()

/Users/satvikpraveen/Desktop/Libraries/PandasPlayground/scripts/agg_utils.py:130: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample("M").agg(metrics_dict).reset_index()


,date,temperature_c,humidity
0,2022-01-31,10.870968,71.806452
1,2022-02-28,14.107143,67.178571
2,2022-03-31,14.096774,61.741935
3,2022-04-30,12.900000,67.866667
4,2022-05-31,18.064516,64.967742


In [17]:
# Save Aggregated Versions
utils_io.save_csv(condition_stats, "../assets/weather_condition_stats.csv")
utils_io.save_csv(weather_monthly, "../assets/weather_monthly_summary.csv")

## Bank Loan Data (loan_final.csv)

In [18]:
# Total Loan Amount by Loan Purpose
loan_by_purpose = (
    loan_df
    .pipe(agg_utils.groupby_summary,
          group_col="loan_purpose",
          agg_dict={
              "loan_amount": "sum",
              "income": "mean"
          })
    .rename(columns={"loan_amount": "total_loan", "income": "avg_income"})
)

# Approval Rate by Purpose
approval_by_purpose = (
    loan_df
    .groupby("loan_purpose")
    .agg(approval_rate=("approved", lambda x: (x == "yes").mean()))
    .reset_index()
)

# Merge Both
loan_summary = pd.merge(loan_by_purpose, approval_by_purpose, on="loan_purpose")

# Display
loan_summary.head()

,loan_purpose,total_loan,avg_income,approval_rate
0,business,69926887,87644.328580,0.496732
1,car,70170581,86448.031063,0.495818
2,education,69328270,88096.910832,0.511670
3,home,67801523,86561.021197,0.466334
4,medical,67047307,87480.750000,0.498197


In [19]:
# Create age brackets
loan_df["age_bracket"] = pd.cut(
    loan_df["age"],
    bins=[20, 30, 40, 50, 60, 70],
    right=False
)

loan_by_age = (
    loan_df
    .pipe(agg_utils.groupby_summary,
          group_col="age_bracket",
          agg_dict={
              "loan_amount": "mean"
          })
    .rename(columns={"loan_amount": "avg_loan_amount"})
)

# Add approval rate
approval_by_age = (
    loan_df
    .groupby("age_bracket")
    .agg(approval_rate=("approved", lambda x: (x == "yes").mean()))
    .reset_index()
)

# Merge both
loan_age_summary = pd.merge(loan_by_age, approval_by_age, on="age_bracket")

# Display
loan_age_summary

/Users/satvikpraveen/Desktop/Libraries/PandasPlayground/scripts/agg_utils.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df.groupby(group_col).agg(agg_dict)
/var/folders/23/phyj43kx4jz5gywlx6__9_kc0000gn/T/ipykernel_55426/2741313085.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("age_bracket")


,age_bracket,avg_loan_amount,approval_rate
0,"[20, 30)",41259.467899,0.496595
1,"[30, 40)",41497.464334,0.503813
2,"[40, 50)",41780.420533,0.484966
3,"[50, 60)",40957.685036,0.491935
4,"[60, 70)",41600.538528,0.481385


In [20]:
# Save to CSV
utils_io.save_csv(loan_by_purpose, "../assets/loan_agg_by_purpose.csv")

utils_io.save_csv(loan_age_summary, "../assets/loan_agg_by_age.csv")

## COVID Dataset (covid_final.csv)

In [21]:
# Daily Aggregation by Country
covid_country_summary = (
    covid_df
    .pipe(agg_utils.groupby_summary,
          group_col="country",
          agg_dict={
              "new_cases": "sum",
              "new_deaths": "sum",
              "hospitalized": "mean"
          })
    .sort_values("new_cases", ascending=False)
)

In [22]:
# Monthly Summary (Cases and Deaths)
covid_monthly = (
    covid_df
    .pipe(agg_utils.resample_monthly, date_col="date", metrics_dict={
        "new_cases": "sum",
        "new_deaths": "sum"
    })
)

/Users/satvikpraveen/Desktop/Libraries/PandasPlayground/scripts/agg_utils.py:130: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  result = df.resample("M").agg(metrics_dict).reset_index()


In [23]:
# Save Aggregated data
utils_io.save_csv(covid_monthly, "../assets/covid_monthly_summary.csv")
utils_io.save_csv(covid_country_summary, "../assets/covid_country_summary.csv")

## ✅ Summary: Aggregation & Reshaping

In this module, you:

- Performed group-wise aggregations using `.groupby()` and `.agg()`
- Pivoted and melted DataFrames for wide-to-long reshaping
- Used `.stack()` / `.unstack()` for hierarchical transformations
- Applied time-based resampling for monthly aggregations
- Saved grouped outputs for dashboards or reporting

## 📦 Multi-Dataset Aggregation Section

In [24]:
# Load all final regional loan datasets
region_files = list(Path("../assets").glob("loan_final_*.csv"))
loan_dfs = [utils_io.load_csv(file) for file in region_files]
loan_all_regions = pd.concat(loan_dfs, ignore_index=True)

# If region column is missing, derive it from filenames (robust)
if "region" not in loan_all_regions.columns:
    regions = [str(f).split("_")[-1].replace(".csv", "") for f in region_files]
    loan_all_regions["region"] = [r for r in regions for _ in range(len(loan_dfs[0]))]

# Group by region: Avg loan amount, approval rate, count
region_loan_summary = groupby_summary(
    df=loan_all_regions,
    group_col="region",
    agg_dict={
        "loan_amount": "mean",
        "approved": lambda x: (x == "yes").mean(),
        "customer_id": "count"
    }
)

# Display
region_loan_summary

,region,loan_amount,approved,customer_id
0,east,41405.8308,0.4927,10000
1,north,41405.8308,0.4927,10000
2,south,41405.8308,0.4927,10000
3,west,41405.8308,0.4927,10000


In [25]:
# Load monthly aggregated datasets
weather_monthly = utils_io.load_csv("../assets/weather_monthly_summary.csv")
covid_monthly = utils_io.load_csv("../assets/covid_monthly_summary.csv")

# Ensure date columns are datetime
weather_monthly["date"] = pd.to_datetime(weather_monthly["date"])
covid_monthly["date"] = pd.to_datetime(covid_monthly["date"])

# Inner join on 'date' to analyze common months
merged_trends = pd.merge(weather_monthly, covid_monthly, on="date", how="inner")

# Display
merged_trends.head()

,date,temperature_c,humidity,new_cases,new_deaths
0,2022-01-31,10.870968,71.806452,15324,316
1,2022-02-28,14.107143,67.178571,13983,300
2,2022-03-31,14.096774,61.741935,15614,292
3,2022-04-30,12.900000,67.866667,14850,292
4,2022-05-31,18.064516,64.967742,15657,309


In [26]:
# Save Combined Outputs
utils_io.save_csv(region_loan_summary, ASSETS_DIR / "agg_loan_by_region.csv")
utils_io.save_csv(merged_trends, ASSETS_DIR / "agg_merged_weather_covid_monthly.csv")

## 📦 Multi-Dataset Aggregation Summary

In this section, we:

- Combined all multi-region loan data into a single frame
- Aggregated region-level metrics (loan amount, approval rate)
- Merged monthly weather and COVID data to support joint time series analysis
- Saved unified outputs for downstream dashboards or ML prep

📦 These outputs can now power interactive dashboards (e.g., Streamlit), merged ML pipelines, or rich correlation studies.


📦 Next: We’ll move into rolling metrics, smoothing, and window functions in `04_merging_joining.ipynb` or `04_window_operations.ipynb`.